In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow import keras
import numpy as np

base_model = keras.applications.VGG19(
    weights = 'imagenet',
    input_shape= (224,224,3),
    include_top = False
)

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
#Freeze base model
base_model.trainable = False

In [ ]:
#Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

#Separately from setting trainable on the model, we set training to false
x = base_model(inputs, training = False)

#Add pooling layer or Flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

#Add final dense layer
outputs = keras.layers.Dense(units=7, activation = 'softmax')(x)

#Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 7)                 3591      
                                                                 
Total params: 20,027,975
Trainable params: 3,591
Non-trainable params: 20,024,384
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=45,
    shear_range = 0.2,
    zoom_range = 0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,

)

# New Section

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('/content/drive/MyDrive/Food Classification/train_noise_and_noise_rot15_flipv', target_size = (224,224),color_mode='rgb',class_mode="categorical")
# load and iterate validation dataset
valid_it=datagen.flow_from_directory('/content/drive/MyDrive/Food Classification/validation',target_size=(224,224),shuffle=False,color_mode='rgb',class_mode="categorical")

Found 2619 images belonging to 7 classes.
Found 216 images belonging to 7 classes.


In [ ]:
model.fit(train_it,
          validation_data=valid_it, 
          steps_per_epoch=train_it.samples/train_it.batch_size, 
          validation_steps=valid_it.samples/valid_it.batch_size, 
          epochs=20)

Epoch 1/20
81/81 [==============================] - 730s 9s/step - loss: 1.8449 - accuracy: 0.2848 - val_loss: 1.6667 - val_accuracy: 0.4444
Epoch 2/20
81/81 [==============================] - 80s 981ms/step - loss: 1.7062 - accuracy: 0.3826 - val_loss: 1.5160 - val_accuracy: 0.5370
Epoch 3/20
81/81 [==============================] - 80s 978ms/step - loss: 1.6116 - accuracy: 0.4387 - val_loss: 1.4013 - val_accuracy: 0.5185
Epoch 4/20
40/81 [=============>................] - ETA: 36s - loss: 1.5551 - accuracy: 0.4484

In [ ]:
#Unfreeze the base model
base_model.trainable = True

#Compile the model with a law learning rate
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate = 0.00001),
loss = keras.losses.CategoricalCrossentropy(from_logits = True),metrics = [keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_it,validation_data=valid_it, steps_per_epoch=train_it.samples/train_it.batch_size, validation_steps=valid_it.samples/valid_it.batch_size, epochs=10)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


81/81 [==============================] - 114s 1s/step - loss: 0.8782 - categorical_accuracy: 0.6831 - val_loss: 0.5085 - val_categorical_accuracy: 0.8426
Epoch 2/10
81/81 [==============================] - 102s 1s/step - loss: 0.5923 - categorical_accuracy: 0.7824 - val_loss: 0.4098 - val_categorical_accuracy: 0.8657
Epoch 3/10
81/81 [==============================] - 102s 1s/step - loss: 0.4552 - categorical_accuracy: 0.8328 - val_loss: 0.3259 - val_categorical_accuracy: 0.8750
Epoch 4/10
81/81 [==============================] - 102s 1s/step - loss: 0.3743 - categorical_accuracy: 0.8614 - val_loss: 0.2516 - val_categorical_accuracy: 0.9167
Epoch 5/10
81/81 [==============================] - 102s 1s/step - loss: 0.3484 - categorical_accuracy: 0.8744 - val_loss: 0.2596 - val_categorical_accuracy: 0.8889
Epoch 6/10
81/81 [==============================] - 102s 1s/step - loss: 0.2900 - categorical_accuracy: 0.8939 - val_loss: 0.2568 - val_categorical_accuracy: 0.9074
Epoch 7/10
81/81 [===

In [ ]:

model.evaluate(valid_it, steps = valid_it.samples/valid_it.batch_size)

6/6 [==============================] - 7s 997ms/step - loss: 0.2419 - categorical_accuracy: 0.9213


[0.24188221991062164, 0.9212962985038757]

In [ ]:
model.save("CNN_VGG19_")

INFO:tensorflow:Assets written to: CNN_VGG19_/assets


In [ ]:
!cp -r "/content/CNN_VGG19_" "/content/drive/MyDrive/Data"

In [ ]:
test_steps_per_epoch = np.math.ceil(valid_it.samples / valid_it.batch_size)
probabilities = model.predict(valid_it, steps=test_steps_per_epoch)
y_pred = np.argmax(probabilities, axis=1)
true_classes = valid_it.classes
class_labels = list(valid_it.class_indices.keys())
import sklearn.metrics as metrics
report = metrics.classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

              precision    recall  f1-score   support

     Class 1       1.00      0.90      0.95        21
     Class 2       0.88      1.00      0.93        43
     Class 3       0.92      0.96      0.94        25
     Class 4       0.85      0.94      0.89        18
     Class 5       0.92      0.92      0.92        38
     Class 6       1.00      0.83      0.91        35
     Class 7       1.00      0.97      0.99        36

    accuracy                           0.94       216
   macro avg       0.94      0.93      0.93       216
weighted avg       0.94      0.94      0.94       216



In [ ]:


y_pred = np.argmax(probabilities, axis=1)
y_pred

array([2, 0, 5, 5, 0, 1, 5, 5, 1, 5, 5, 2, 1, 0, 3, 4, 6, 5, 4, 5, 4, 6,
       1, 4, 3, 4, 6, 4, 2, 6, 1, 6, 4, 6, 5, 5, 5, 1, 1, 6, 6, 2, 4, 1,
       4, 4, 5, 0, 1, 4, 1, 4, 4, 1, 1, 0, 1, 2, 1, 4, 2, 2, 6, 2, 5, 4,
       6, 6, 6, 1, 4, 6, 5, 4, 5, 4, 5, 2, 5, 0, 6, 0, 5, 2, 6, 5, 0, 6,
       1, 4, 1, 3, 1, 5, 6, 4, 3, 1, 6, 5, 4, 0, 4, 6, 1, 2, 5, 2, 4, 5,
       6, 5, 6, 4, 4, 4, 2, 2, 4, 2, 6, 5, 6, 0, 6, 6, 5, 1, 1, 5, 4, 4,
       4, 0, 0, 1, 0, 2, 4, 2, 6, 6, 1, 4, 2, 5, 2, 1, 5, 2, 1, 1, 1, 0,
       1, 0, 5, 2, 1, 0, 1, 4, 6, 2, 2, 4, 5, 0, 6, 1, 1, 6, 4, 4, 2, 0,
       5, 4, 0, 6, 6, 5, 1, 5, 6, 5, 1, 6, 0, 5, 5, 4, 1, 4, 2, 2, 5, 2,
       4, 5, 5, 1, 5, 0, 1, 5, 5, 0, 2, 6, 5, 5, 4, 4, 1, 1])